```
QUERYNAME = CholesterolMSMS;
DEFINE PR = 'C[41..49] H[30..200] O[2] N[1]' WITH DBR = (4.5,10.5), CHG = +1;
DEFINE FR ='C[27] H[45]' WITH DBR = (1,8), CHG = +1;

IDENTIFY

	PR IN MS1+ AND
	FR in MS2+

SUCHTHAT
	(avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
	PR.chemsc[C]/PR.chemsc[db] >= 4.6

REPORT 

	SPECIE = "CE %d:%d" % "((PR.chemsc)[C] - 27, (PR.chemsc)[db] - 4.5)";  
	CLASS = "CE";
	MASS = PR.mass;
	ISOBARIC = PR.isobaric;
	CHEMSC = PR.chemsc;
	ERRppm = "%2.2f" % "(PR.errppm)";
	FRERRppm = "%2.2f" % "(FR.errppm)";
	INT = FR.intensity;
	QUALA = PR.intensity;
```

In [23]:
from targets import Targets_util
from MS_reader import SpectraUtil
from mfql_Parser import fromFile 

In [24]:
from chemParser import txt2dict

In [25]:
filename = '..\\test_resources\\small_test\\170213_CE_pos_MSMS.mfql'
mfql_dict = fromFile(filename)

In [26]:
mfql_dict

{'scriptname': 'CholesterolMSMS',
 'variables': [Var(id='PR', object=ElementSeq(txt='C[41..49] H[30..200] O[2] N[1]'), Options={'dbr': (4.5, 10.5), 'chg': 1}),
  Var(id='FR', object=ElementSeq(txt='C[27] H[45]'), Options={'dbr': (1, 8), 'chg': 1})],
 'identification': Evaluable(operation='AND', term_1=Evaluable(operation='IN', term_1='PR', term_2='MS1+'), term_2=Evaluable(operation='IN', term_1='FR', term_2='MS2+')),
 'suchthat': Evaluable(operation='AND', term_1=Evaluable(operation='AND', term_1=Evaluable(operation='>=', term_1=Evaluable(operation='/', term_1=Func(func='avg', on=[Obj(p_rule='p_withAttr_id', p_values=['FR', '.', 'intensity'])]), term_2=Func(func='avg', on=[Obj(p_rule='p_withAttr_id', p_values=['PR', '.', 'intensity'])])), term_2=0.1), term_2=Evaluable(operation='<=', term_1=Evaluable(operation='/', term_1=Func(func='avg', on=[Obj(p_rule='p_withAttr_id', p_values=['FR', '.', 'intensity'])]), term_2=Func(func='avg', on=[Obj(p_rule='p_withAttr_id', p_values=['PR', '.', 'i

In [27]:
mfql_dict['scriptname']

'CholesterolMSMS'

In [28]:
def var2Target(var):
    elements = txt2dict(var1.object.txt)
    target = Targets_util(elements)
    dbrs = var.Options.get('dbr',(None, None))
    target.set_dbr(*dbrs)
    return target

In [29]:
targets = [var2Target(var) for var in mfql_dict['variables']]


NameError: name 'var1' is not defined

In [0]:
filename = '..\\test_resources\\small_test\\190321_Serum_Lipidextract_368723_01.mzML'
spectraUtil = SpectraUtil.fromFile(filename, test_sample=True)

In [0]:
MS1 = spectraUtil
MS2 = spectraUtil.get_reset_copy()
MS1.set_mode()
MS1.set_ms_level()
MS2.set_mode()
MS2.set_ms_level(2)


In [0]:
PR = targets[0]
FR = targets[1]
MS1_match = MS1.get_nearest(PR._df)
MS2_match = MS2.get_nearest(FR._df)

## get only the nearest peak in each scan

In [0]:
MS1_match.shape

In [0]:
MS1_match = Targets_util.set_max_daltons(MS1_match)
MS2_match = Targets_util.set_max_daltons(MS2_match)

In [0]:
MS1_match.columns

In [0]:
MS1_match.shape

In [0]:
all_match = Targets_util.makeAllCombo(MS1_match, MS2_match)

In [0]:
all_match.columns

In [0]:
# # (avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
# 	PR.chemsc[C]/PR.chemsc[db] >= 4.6

ST = all_match.query('FR_i/PR_i >= 0.1 and FR_i/PR_i <= 10 and PR_C/PR_dbr >= 4.6')

# make it a summary like the current lx, 
# one entry one line


In [0]:
pr_df, fr_df = Targets_util.devideAllCombo(ST)

In [0]:
pr_df[pr_df.PR_chem== 'C41 H76 O2 N1 P1']

In [0]:
sum_df = Targets_util.summaryDF(pr_df, quantile=1)
sum_df

In [0]:
plt = Targets_util.lollipop_plot(pr_df.PR_m, pr_df.PR_i)
plt.show()
plt = Targets_util.lollipop_plot(fr_df.FR_m, fr_df.FR_i)
plt.show()

In [0]:
MFQL_util.showAll_lollipop(pr_df, sample = False)

# in the XML annotate with 'selectedIonList'